In [1]:
# Core 
import datetime
import os
import glob
import tempfile
import shutil
import sys
import pickle
import warnings

# Analysis 
import xarray as xr
import numpy as np
import pandas as pd

# Debugging 
import pdb, traceback
%load_ext line_profiler

# Radar Tools
import pyart
import tint
from tint import animate


## You are using the Python ARM Radar Toolkit (Py-ART), an open source
## library for working with weather radar data. Py-ART is partly
## supported by the U.S. Department of Energy as part of the Atmospheric
## Radiation Measurement (ARM) Climate Research Facility, an Office of
## Science user facility.
##
## If you use this software to prepare a publication, please cite:
##
##     JJ Helmus and SM Collis, JORS 2016, doi: 10.5334/jors.119



/g/data3/hh5/public/apps/miniconda3/envs/analysis3-19.04/lib/python3.6/site-packages/pyart/graph/cm.py:104: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if 'red' in spec:
/g/data3/hh5/public/apps/miniconda3/envs/analysis3-19.04/lib/python3.6/site-packages/pyart/graph/cm_colorblind.py:32: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if 'red' in spec:


In [2]:
def CPOL_files_from_datetime_list(datetimes):
    print('Gathering files.')
    base = '/g/data/rr5/CPOL_radar/CPOL_level_1b/GRIDDED/GRID_70km_1000m/'
    filenames = []
    for i in range(len(datetimes)):
        year = str(datetimes[i])[0:4]
        month = str(datetimes[i])[5:7]
        day = str(datetimes[i])[8:10]
        hour = str(datetimes[i])[11:13]
        minute = str(datetimes[i])[14:16]
        filename = (base + '{0}/{0}{1}{2}/'.format(year, month, day) 
                    + 'CPOL_{0}{1}{2}'.format(year, month, day)
                    + '_{}{}_GRIDS_1000m.nc'.format(hour, minute))
        if os.path.isfile(filename):
            filenames.append(filename)
    
    return sorted(filenames), datetimes[0], datetimes[-1]
    
def CPOL_files_from_TINT_obj(tracks_obj, uid):
    datetimes = tracks_obj.system_tracks.xs(uid, level='uid')
    datetimes = datetimes.reset_index(level='time')['time']
    datetimes = list(datetimes.values)
    [files, start_date, end_date] = CPOL_files_from_datetime_list(datetimes)
    
    return files, start_date, end_date

In [3]:
with open('/g/data/w40/esh563/CPOL_analysis/TINT_tracks/tracks_obj_45403030.pkl', 'rb') as f:
    tracks_obj = pickle.load(f)

In [ ]:
# Store filenames for an example day of data
base = '/g/data/rr5/CPOL_radar/CPOL_level_1b/GRIDDED/GRID_70km_1000m/'

filenames = CPOL_files_from_datetime_list(np.arange(np.datetime64('2010-11-01'), 
                                          np.datetime64('2011-04-01'), 
                                          np.timedelta64(10, 'm')))[0]
# Generate grid generator 
# Note generators produce iterators
# These are alternative to using lists and looping
grids = (pyart.io.read_grid(fn, include_fields = 'reflectivity') 
         for fn in filenames)

# Define settings for tracking
settings = {
    'MIN_SIZE' : [15, 15, 15, 15],
    'FIELD_THRESH' : [45, 40, 30, 30],
    'ISO_THRESH' : [25, 25, 10, 10],
    'GS_ALT' : 1500,
    'LEVELS' : np.array(
        [[0, 2500], 
         [2500, 5000],
         [5000, 7500],
         [7500, 10000]]
    ),
    'TRACK_INTERVAL' : 0,
}

# Calculate high and low level tracks
tracks_obj  = tint.Cell_tracks()

for parameter in ['MIN_SIZE', 'FIELD_THRESH', 'GS_ALT', 'LEVELS', 
                  'TRACK_INTERVAL', 'ISO_THRESH'
                 ]:
    tracks_obj.params[parameter] = settings[parameter]

# Calculate tracks
tracks_obj.get_tracks(grids)

# Create directory for figures
dt=str(datetime.datetime.now())[0:-7]
dt=dt.replace(" ", "_")
dt=dt.replace(":", "_")
dt=dt.replace("-", "")  

out_file_name = ('/g/data/w40/esh563/CPOL_analysis/TINT_tracks/'
                 + 'tracks_obj_{}.pkl'.format(dt))

with open(out_file_name, 'wb') as f:
    pickle.dump(tracks_obj, f)
    
# if __name__ == '__main__':
#     try:
#         test()
#     except:
#         extype, value, tb = sys.exc_info()
#         traceback.print_exc()
#         pdb.post_mortem(tb)

Gathering files.


In [33]:
lon_min = 131.0 - 0.8
lon_max = 131.0 + 0.8

lat_max = -12.2 + 0.8
lat_min = -12.2 - 0.8

# [filenames, start_time, end_time] = CPOL_files_from_datetime_list(np.arange(np.datetime64('2006-02-09'), 
#                                                                 np.datetime64('2006-02-13'), 
#                                                                 np.timedelta64(10, 'm')))

filenames, start_time, end_time = CPOL_files_from_TINT_obj(tracks_obj, '10')

# Generate grid generator 
# Note generators produce iterators
# These are alternative to using lists and looping
grids = (pyart.io.read_grid(fn, include_fields = 'reflectivity')
         for fn in filenames)

# Create directory for figures
dt=str(datetime.datetime.now())[0:-7]
dt=dt.replace(" ", "_")
dt=dt.replace(":", "_")
dt=dt.replace("-", "")    

base_path = '/g/data/w40/esh563/CPOL_analysis/figures/TINT/'
out_path = base_path + dt + '_tint_CPOL_anim'

animate(tracks_obj, grids, out_path,
        start_datetime = start_time,
        end_datetime = end_time,
        lat_lines=np.arange(lat_min, lat_max, .1),
        lon_lines=np.arange(lon_min, lon_max, .1),
        tracers=True, keep_frames=False, dpi=100, alt=2500)

Gathering files.
Animating from 2011-01-01T10:20:03.000000000 to 2011-01-01T12:10:03.000000000.
Plotting scan at 2011-01-01T12:10:03.
Creating GIF - may take a few minutes.


In [31]:
# Let's filter by tilt direction and magnitude
forward_tilt = ((-45 <= tracks_obj.system_tracks['sys_rel_tilt_dir'])
                & (tracks_obj.system_tracks['sys_rel_tilt_dir'] <= 45))
backward_tilt = ((-45 >= tracks_obj.system_tracks['sys_rel_tilt_dir'])
                | (tracks_obj.system_tracks['sys_rel_tilt_dir'] >= 45))
left_tilt = ((45 <= tracks_obj.system_tracks['sys_rel_tilt_dir'])
                & (tracks_obj.system_tracks['sys_rel_tilt_dir'] <= 135))
right_tilt = ((-135 <= tracks_obj.system_tracks['sys_rel_tilt_dir'])
                & (tracks_obj.system_tracks['sys_rel_tilt_dir'] <= -45))
sig_tilt_mag = (tracks_obj.system_tracks['tilt_mag'] >= 2000)

In [37]:
# Let's find the cells that were tracked for the most frames
tracks_obj.system_tracks.groupby(level='uid').size().sort_values(ascending=False)[:20]

uid
10    12
19     9
15     9
21     6
29     6
16     6
3      5
20     4
7      3
8      3
36     3
25     2
0      2
34     2
35     2
12     2
5      2
1      2
13     1
17     1
dtype: int64

In [14]:
tracks_obj.system_tracks['sys_rel_tilt_dir'].xs('570', level='uid')

scan   time               
10892  2015-01-31 19:30:02        NaN
10893  2015-01-31 19:40:02    176.921
10894  2015-01-31 19:50:02   -160.672
10895  2015-01-31 20:00:01    -22.937
10896  2015-01-31 20:10:06    -13.229
10897  2015-01-31 20:20:02    -86.087
10898  2015-01-31 20:30:02     -9.632
10899  2015-01-31 20:40:02    -15.275
10900  2015-01-31 20:50:02   -130.088
10901  2015-01-31 21:00:02   -171.258
10902  2015-01-31 21:10:06    -66.726
10903  2015-01-31 21:20:02   -136.492
10904  2015-01-31 21:30:02    -75.989
10905  2015-01-31 21:40:03    123.494
10906  2015-01-31 21:50:03   -163.507
10907  2015-01-31 22:00:02   -119.481
10908  2015-01-31 22:10:06     93.505
10909  2015-01-31 22:20:01   -103.949
10910  2015-01-31 22:30:02    -99.390
10911  2015-01-31 22:40:01   -171.813
10912  2015-01-31 22:50:02   -128.295
10913  2015-01-31 23:00:02   -164.600
10914  2015-01-31 23:10:06   -109.363
10915  2015-01-31 23:20:02   -109.276
10916  2015-01-31 23:30:02    113.234
10917  2015-01-31 23:40

In [38]:
uid = '19'

filenames, start_time, end_time = CPOL_files_from_TINT_obj(tracks_obj, uid)

grids = (pyart.io.read_grid(fn) for fn in filenames)  # refresh grid generator

# Create directory for figures
dt=str(datetime.datetime.now())[0:-7]
dt=dt.replace(" ", "_")
dt=dt.replace(":", "_")
dt=dt.replace("-", "")

base_path = '/g/data/w40/esh563/CPOL_analysis/figures/TINT/'
out_path = base_path + dt + '_tint_CPOL_lagrangian'
animate(tracks_obj, grids, out_path, style='lagrangian', uid=uid, alt=3000,
        keep_frames=False, dpi=100)

Gathering files.
Animating 9 frames
Plotting frame at 2011-01-03T02:00:05
Creating GIF - may take a few minutes.
